In [ ]:
# !pip install selenium

In [ ]:
# !pip install beautifulsoup4

tqdm은 없을 가능성이 크니까 install 되어있는지 한 번 돌려보세요.

In [ ]:
# !pip install tqdm

In [30]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
from tqdm import tqdm
import json

# 웹 드라이버 선택
driver = webdriver.Chrome()

# 웹 드라이버 실행
driver.get('https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query=%EC%84%B1%EC%88%98+%ED%8C%9D%EC%97%85')


"""
셀레니움으로 사이트 주소 리스트를 저장하는 과정입니다.
get_site 함수는 사이트 각 설정 값 적용 및 게시물별 주소를 저장합니다.
"""
# 사이트 주소 담을 리스트
site_list = []

# 옵션버튼 최초 1회만 클릭하기 위한 변수
option_button = 0

def get_site(year, size):
    # 옵션버튼 최초 1회만 클릭하도록 설정
    global option_button
    if option_button == 0:
        driver.find_element(By.CSS_SELECTOR, 'i.spnew.ico_filter_arr').click()
        option_button += 1
        driver.implicitly_wait(3)

    # 기간 선택 화살표 클릭
    driver.find_element(By.CSS_SELECTOR, 'i.spnew.ico_check').click()
    driver.implicitly_wait(3)

    # 시작하는 날짜 선택
    driver.find_element(By.CSS_SELECTOR, 'a.spnew_bf.ico_calendar._start_trigger').click()
    driver.implicitly_wait(3)

    # year년 1월 1일 선택
    driver.find_element(By.XPATH, f'//div[@class="group_select _list_root"]//strong[contains(text(), "년(Year)")]/following-sibling::div[@class="select_cont"]//ul[@class="lst_item _ul"]/li[@data-value="{year}"]').click()
    driver.implicitly_wait(3)
    driver.find_element(By.XPATH, '//div[@class="group_select _list_root"]//strong[contains(text(), "월(Month)")]/following-sibling::div[@class="select_cont"]//ul[@class="lst_item _ul"]/li[@data-value="1"]').click()
    driver.implicitly_wait(3)
    driver.find_element(By.XPATH, '//div[@class="group_select _list_root"]//strong[contains(text(), "일(Day)")]/following-sibling::div[@class="select_cont"]//ul[@class="lst_item _ul"]/li[@data-value="1"]').click()
    driver.implicitly_wait(3)

    # 끝나는 날짜 선택
    driver.find_element(By.CSS_SELECTOR, 'a.spnew_bf.ico_calendar._end_trigger').click()
    driver.implicitly_wait(3)

    # year년 12월 31일 선택
    driver.find_element(By.XPATH, f'//div[@class="group_select _list_root"]//strong[contains(text(), "년(Year)")]/following-sibling::div[@class="select_cont"]//ul[@class="lst_item _ul"]/li[@data-value="{year}"]').click()
    driver.implicitly_wait(3)
    driver.find_element(By.XPATH, '//div[@class="group_select _list_root"]//strong[contains(text(), "월(Month)")]/following-sibling::div[@class="select_cont"]//ul[@class="lst_item _ul"]/li[@data-value="12"]').click()
    driver.implicitly_wait(3)
    driver.find_element(By.XPATH, '//div[@class="group_select _list_root"]//strong[contains(text(), "일(Day)")]/following-sibling::div[@class="select_cont"]//ul[@class="lst_item _ul"]/li[@data-value="31"]').click()
    driver.implicitly_wait(3)
    
    # 적용 버튼 클릭
    driver.find_element(By.CSS_SELECTOR, 'button.btn_apply._apply_btn').click()
    driver.implicitly_wait(3)

    # 1000개 이상이 될때까지 스크롤
    while len(driver.find_elements(By.CSS_SELECTOR, 'a.title_link')) < size:
        driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)
        driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)
        driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)
        driver.implicitly_wait(3)

    # 진행 상황 확인을 위해 글로벌 변수화(몰라도 됨)
    global href_list

    # 개별 사이트 주소를 리스트에 저장하고, 1000개 넘어갈 경우 제거
    href_list = [i.get_attribute('href') for i in driver.find_elements(By.CSS_SELECTOR, 'a.title_link')][:size]
    site_list.extend(href_list)

    # 날짜 설정 변경을 위해 가장 상단으로 스크롤 이동
    driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.HOME)
    driver.implicitly_wait(3)


"""
수집된 개별 사이트 주소를 이용해서 게시날짜, 제목, 본문을 저장하는 과정입니다.
개별 사이트는 iframe이라는 형식으로 되어 있습니다.
따라서 셀레니움이 해당사이트에 방문 후 iframe에 진입해야 크롤링이 가능합니다.
""" 
# 본문 담을 리스트
content_list = []

# 년도별 데이터 담을 딕셔너리
year_class = {}

# selenium으로 iframe에 들어가서 본문 긁어오기 함수
def get_content(url):
    # 사이트 방문
    driver.get(url)
    driver.implicitly_wait(3)

    # iframe영역으로 들어가기
    driver.switch_to.frame('mainFrame')
    driver.implicitly_wait(3)

    # 날짜
    dates = driver.find_element(By.CSS_SELECTOR, 'span.se_publishDate.pcol2').text.strip()
    driver.implicitly_wait(3)

    # 제목
    title = driver.find_element(By.CSS_SELECTOR, 'div.se-module.se-module-text.se-title-text').text.strip()
    driver.implicitly_wait(3)

    # 본문
    content = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'div.se-component.se-text.se-l-default')]
    content = ''.join(content).strip().replace('\n', ' ')
    driver.implicitly_wait(3)

    content_dict = {"date" : dates,
                    "title" :title,
                    "content" : content}

    # 년도만 출력
    year = content_dict['date'].split('.')[0]
    
    # {년도 : content_dict}틀 만들기 
    if year not in year_class:
        year_class[year] = []
    
    # 년도별 딕셔너리 형태로 저장
    year_class[year].append(content_dict)
    
    ## iframe 영역 나가기
    # driver.switch_to.default_content() 


"""
앞서 만든 get_site함수와 get_content함수를 통합한 함수입니다.
해당 함수에 
year_list = 수집할 연도 리스트
year_size = 연도별 글 개수
이 두가지만 입력하면 자동으로 돌아갑니다.
"""

# 연도별, 주소 수집 함수
def get_json(year_list, year_size):
    for i in tqdm(year_list):
        get_site(i, year_size)
        print(f'{i}년 사이트 주소 : {len(href_list)}개 수집')
        print('-'*50)
        print()

    print(f'총 사이트 주소 개수 : {len(site_list)}개')
    print('본문 수집 시작')

    # 본문 수집
    for site in tqdm(site_list):
        get_content(site)

    content_list.append(year_class)

    # 컨텐츠 목록 json파일로 만들기
    with open ('content_list.json', 'w', encoding='utf-8') as f:
        json.dump(content_list, f, ensure_ascii=False, indent=4) 


"""
사용할 때 변경할 부분은 이 아래가 전부입니다.
year_list와 year_size만 설정하세요.
"""
# 수집할 연도 리스트(2020~2024년)
year_list = [i for i in range(2020, 2025)]

# 수집할 연도별 글 개수
year_size = 1000

# 통합 함수 실행
get_site(year_list, year_size)


driver.close() # 해당 브라우저만 닫기
# driver.quit()  # 전체 브라우저 모두 닫기


 50%|█████     | 1/2 [00:05<00:05,  5.20s/it]

2022년  : 40개 수집

--------------------------------------------------


100%|██████████| 2/2 [00:08<00:00,  4.21s/it]


2023년  : 40개 수집

--------------------------------------------------
총 사이트 주소 개수 : 80개

--------------------------------------------------
본문 수집 시작


100%|██████████| 80/80 [05:36<00:00,  4.21s/it]
